In [1]:
from demo.simple_tokenizer import SimpleTokenizer as _Tokenizer
from demo.viclip import ViCLIP
import torch
import numpy as np
import cv2

/localhome/local-haowan/miniconda3/envs/viclip/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
clip_candidates = {'viclip':None, 'clip':None}

def get_clip(name='viclip'):
    global clip_candidates
    m = clip_candidates[name]
    if m is None:
        if name == 'viclip':
            tokenizer = _Tokenizer()
            print('loading ViCLIP model...')
            vclip = ViCLIP(tokenizer)
            # m = vclip
            m = (vclip, tokenizer)
        else:
            raise Exception('the target clip model is not found.')
    return m

def get_text_feat_dict(texts, clip, tokenizer, text_feat_d={}):
    for t in texts:
        feat = clip.get_text_features(t, tokenizer, text_feat_d)
        text_feat_d[t] = feat
    return text_feat_d

def get_vid_feat(frames, clip):
    return clip.get_vid_features(frames)

def _frame_from_video(video):
    while video.isOpened():
        success, frame = video.read()
        if success:
            yield frame
        else:
            break

In [8]:
video = cv2.VideoCapture('demo/example2.mp4')
frames = [x for x in _frame_from_video(video)]
print(frames[0].shape)
print(len(frames))

(720, 1280, 3)
275


In [9]:
v_mean = np.array([0.485, 0.456, 0.406]).reshape(1,1,3)
v_std = np.array([0.229, 0.224, 0.225]).reshape(1,1,3)

def normalize(data):
    return (data/255.0-v_mean)/v_std

def frames2tensor(vid_list, fnum=8, target_size=(224, 224), device=torch.device('cuda')):
    assert(len(vid_list) >= fnum)
    step = len(vid_list) // fnum
    vid_list = vid_list[::step][:fnum]
    vid_list = [cv2.resize(x[:,:,::-1], target_size) for x in vid_list]
    vid_tube = [np.expand_dims(normalize(x), axis=(0, 1)) for x in vid_list]
    vid_tube = np.concatenate(vid_tube, axis=1)
    vid_tube = np.transpose(vid_tube, (0, 1, 4, 2, 3))
    vid_tube = torch.from_numpy(vid_tube).to(device, non_blocking=True).float()
    return vid_tube

def retrieve_text(frames, texts, name='viclip', topk=5, device=torch.device('cuda')):
    clip, tokenizer = get_clip(name)
    clip = clip.to(device)
    print(type(frames))
    frames_tensor = frames2tensor(frames, device=device)
    print(frames_tensor.shape)
    vid_feat = get_vid_feat(frames_tensor, clip)
    print(vid_feat.shape)
    print(vid_feat.sum(dim=-1))

    text_feat_d = {}
    text_feat_d = get_text_feat_dict(texts, clip, tokenizer, text_feat_d)
    text_feats = [text_feat_d[t] for t in texts]
    text_feats_tensor = torch.cat(text_feats, 0)
    print(text_feats_tensor.shape)

    probs, idxs = clip.get_predict_label(vid_feat, text_feats_tensor, top=topk)

    ret_texts = [texts[i] for i in idxs.numpy()[0].tolist()]
    return ret_texts, probs.numpy()[0]

In [23]:
text_candidates = [
    "A playful dog and its owner wrestle in the snowy yard, chasing each other with joyous abandon.",
    "A man in a gray coat walks through the snowy landscape, pulling a sleigh loaded with toys.",
    "A man in a gray sweater plays fetch with his dog in the snowy yard, throwing a toy and watching it run.",
    "Lebron James dribbles the basketball down the court, fakes out his defender, and scores a layup.",
    "Lebron James makes a slamp dunk in a fast break.",
    "Lebron James of the Lakers makes a slamp dunk in a fast break.",
]

texts, probs = retrieve_text(frames, text_candidates, name='viclip', topk=5)

for t, p in zip(texts, probs):
    print(f'text: {t} ~ prob: {p:.4f}')

loading ViCLIP model...
building vision encoder ...
building text encoder ...
<class 'list'>
torch.Size([1, 8, 3, 224, 224])
torch.Size([1, 768])
tensor([1.5614], device='cuda:0')
torch.Size([6, 768])
text: Lebron James of the Lakers makes a slamp dunk in a fast break. ~ prob: 0.9544
text: Lebron James dribbles the basketball down the court, fakes out his defender, and scores a layup. ~ prob: 0.0232
text: Lebron James makes a slamp dunk in a fast break. ~ prob: 0.0224
text: A man in a gray sweater plays fetch with his dog in the snowy yard, throwing a toy and watching it run. ~ prob: 0.0000
text: A man in a gray coat walks through the snowy landscape, pulling a sleigh loaded with toys. ~ prob: 0.0000
